In [1]:
import os
import numpy as np
from collections import Counter

# from tabulate import tabulate
from tqdm import tqdm
# import keras
# import keras.backend as K

# load data

In [2]:
def load_label(filename):
    label = dict()

    with open(filename) as file:
        for line in file:
            spl = line.strip().split(' ')
            label[spl[0]] = []
            for ph in spl[1:]:
                label[spl[0]].append(ph)
                
    return label

In [3]:
def load_prediction(filename, onebest=False):
    pred = dict()
    nbest = dict()
    best = 0
    with open(filename) as file:
        lines = file.readlines()
        change = False
        name = ''
        for line in lines:
            line = line.strip()
            if(line != ''):
                if(line[0] in ['F', 'M', 'D', 'H']):
                    change = True
                    name = line
#                     pred[name] = (set(),set(),set())       
                    pred[name] = (Counter(), Counter(), Counter())
                    nbest[name] = dict()
                    best = 1
                elif(onebest):
                    if(pred[name] == (set(), set(), set())):    
                        target = line.split(' ')[1:]
                        post = post_processing(target)
                        pred[name] = (pred[name][0] | set(post[0]),
                                      pred[name][1] | set(post[1]),
                                      pred[name][2] | set(post[2]))
                else:
                        target = line.split(' ')[1:]
                        post = post_processing(target)
#                         pred[name] = (pred[name][0] | set(post[0]),
#                                       pred[name][1] | set(post[1]),
#                                       pred[name][2] | set(post[2]))
                        
                        for ph in post[0]:
                            pred[name][0][ph] += 1
                            nbest[name][ph] = best
                        for ph in post[1]:
                            pred[name][1][ph] += 1
                            nbest[name][ph] = best
                        for ph in post[2]:
                            pred[name][2][ph] += 1
                            nbest[name][ph] = best
                        best += 1

    res = dict()
    for name in sorted(pred):
        res[name] = ['<eps>', '<eps>', '<eps>']
        res[name] = get_representative(pred[name][0], nbest[name]), get_representative(pred[name][1], nbest[name]), \
                    get_representative(pred[name][2], nbest[name])
    return res
#     return pred

In [4]:
def get_representative(pred, nbest):
    res = ''
    nmax = 0
    nbestmax = 31
    for pk in pred:
        if(pred[pk] >= nmax):
            if(pred[pk] == nmax and nbest[pk] > nbestmax):
                break
            res = pk
            nmax = pred[pk]
            nbestmax = nbest[pk]
    return res

# Post processing

In [5]:
def isVow(x):
    return x in ['a','aa', 'i', 'ii', 'v', 'vv', 'u', 'uu', 'e', 'ee',
                'x', 'xx', 'o', 'oo', '@', '@@', 'q', 'qq', 
                'ia', 'iia', 'va', 'vva', 'ua', 'uua']


def isFinal(x):
    return x[-1] == '^'


def isInit(x):
    return not isVow(x) and not isFinal(x)

In [6]:
def post_processing(phones):
    c_i = list() #initial consonant
    v = list() #vowel
    c_f = list() #final consonant
    over_spelling = list()    
    state = 0
    phones = clean(phones)
    for i in range(len(phones)):
        if(state == 0):
            if(isInit(phones[i])): c_i.append(phones[i])
            elif(isVow(phones[i])): state = 1
            else: state = 2
                
        if(state == 1):
            if(isInit(phones[i])): state = 3
            elif(isVow(phones[i])): v.append(phones[i])
            else: state = 2
        
        if(state == 2):
            if(isInit(phones[i])): state = 3
            elif(isVow(phones[i])): state = 3
            else: c_f.append(phones[i])
        
        else: #state = 3
            pass
    
    return c_i, v, c_f
    
    
def remove_interfering(phones, n=1):
    state = None
    skip = 0
    i = 0
    
    while(i < len(phones)):
        ph = phones[i]
#         print(i, state, skip)
        if(state == None):
            state = ph
            skip = 0
        elif(ph != state):
            if(skip < n):
                skip += 1
            else:
                state = None               
                i = i - skip - 1               
        elif(skip > 0):
            for j in range(skip):
                phones[i-j-1] = state
            count = 0
        i += 1
        
    return phones


def remove_duplicate(phones):
    result = []
    for i in range(len(phones)):
        if(len(result) == 0):
            result.append(phones[i])
        elif(phones[i] != result[-1]):
            result.append(phones[i])
    return result


def clean(phones):
    temp = phones
    while True:
        phones = remove_duplicate(remove_interfering(phones))
        if(temp == phones):
            break
        temp = phones
    return phones

# accuracy score

In [7]:
# (a/na + b/nb + c/nc) / 3 
# or
# (a+b+c) / (na+nb+nc) 

def count_correct(text , target, debug=False):
    init, vowel, final = text     
    t_final = None
    n = 3
    if(len(final) == 0): n = 2
    
    if(len(target) == 3): t_init, t_vowel, t_final = target
    else: t_init, t_vowel = target
    
    correct = 0
    if(debug):
        print(text)
        print(target)    
    
    if(t_init in init):
        correct += 1
        if(debug):
            print('init:', end=' ')
        
    if(t_vowel in vowel):
        correct += 1
        if(debug):
            print('vowel:', end=' ')
    
    if(t_final != None and t_final in final):
        correct += 1
        if(debug):
            print('final:', end=' ')
            
    if(t_final == None and len(final) == 0):
        correct += 1 
        if(debug):
            print('final emp:', end= ' ')

    if(debug):
        if(correct > 0): print()
        print(correct, end='\n\n')
    
    return correct, n


def scoring(pred_file='nbest-disabilities-real.txt', lab_file='text-real', debug=False, onebest=False):
    score = 0
    pred = load_prediction(pred_file, onebest)
    label = load_label(lab_file)
    n_pred = 0
    for k in sorted(list(pred.keys())):
        if(debug): print(k)
#         score += count_correct(pred[k], label[k], debug)
#     return score / len(pred)
        c, n = count_correct(pred[k], label[k], debug)     
        score += c
        n_pred += n
    return score / n_pred

In [8]:
scoring(pred_file='30best-disability-dLM-online.txt' , lab_file='text-real')

0.42028985507246375

In [9]:
scoring('30best-dhealthy-dLM-online.txt', 'text-dhealthy')

0.5547445255474452

# distance score

In [10]:
def isVow(x):
    return x in ['a','aa', 'i', 'ii', 'v', 'vv', 'u', 'uu', 'e', 'ee',
                'x', 'xx', 'o', 'oo', '@', '@@', 'q', 'qq', 
                'ia', 'iia', 'va', 'vva', 'ua', 'uua']


def isFinal(x):
    return x[-1] == '^'


def isInit(x):
    return not isVow(x) and not isFinal(x)


# INIT ---------------------------------------------------------------------

    
def get_cluster_cost(x , y):
    cluster_cost = 0
    if(x[-1] in ['r', 'l']):        
        if(not y[-1] in ['r', 'l']):
            cluster_cost += 0.5        
        
    elif(x[-1] == 'w' and len(x) > 1):
        if(not y[-1] == 'w' and len(y) > 1):
            cluster_cost += 1                    
    return cluster_cost
    

def clean_cluster(x):
    if(len(x) > 1):
        if(x[-1] in ['r', 'l']):
            x = x[:-1]
        if(x[-1] == 'w'):
            x = x[-1]
    return x


def initc_score(x,y):
    #REF table: https://en.wikipedia.org/wiki/Thai_language#Initials
    #column: Labial, Alveolar, Palatal, Velar, Gloattal
    #row: Nasal, Plosive_voice, Plosive_tenuis, Plotsive_aspirated, Fricative, Approximant, Trill
    table = [[] for i in range(7)]
    table[0].extend(['m', 'n', '' , 'ng', ''])
    table[1].extend(['b', 'd', '' , '', ''])
    table[2].extend(['p', 't', 'c', 'k', 'z'])
    table[3].extend(['ph', 'th', 'ch', 'kh', ''])
    table[4].extend(['f', 's', '', '', 'h'])
    table[5].extend(['', 'l', 'j', 'w', ''])
    table[6].extend(['', 'r', '', '', ''])
    
    cluster_cost = get_cluster_cost(x, y)
    cluster_cost += get_cluster_cost(y, x)
    
    x = clean_cluster(x)
    y = clean_cluster(y)    
    
    for i in range(7):
        for j in range(5):
            if(x == table[i][j]):
                xi = i; xj = j;
            if(y == table[i][j]):
                yi = i; yj = j;
    
    dist = ((xi-yi)**2 + (xj-yj)**2)**0.5 + cluster_cost
#     return dist
    return min(dist, 4.0)/5.0
    
    
# VOWEL ---------------------------------------------------------------------
    
    
def get_dipthongs_j(x):
    if(x in ['ia', 'iia']): return 0.5
    elif(x in ['va','vva']): return 2.5
    else: return 4.5
    
    
def vow_score(x,y):
    # REF table: https://en.wikipedia.org/wiki/Thai_language#Vowels
    # column: FUS, FUL, BUS, BUL, BRS, BRL
    # F: Front, B: Back, U: Unrounded, R: Rounded, S: Short, L: Long
    # row: High, Mid, Low 

    table = [[] for i in range(3)]
    table[0].extend(['i', 'ii', 'v', 'vv', 'u', 'uu'])
    table[1].extend(['e', 'ee', 'q', 'qq', 'o', 'oo'])
    table[2].extend(['x', 'xx', 'a', 'aa', '@', '@@'])

    dipthongs = ['ia', 'iia', 'va', 'vva', 'ua', 'uua']
   
    xi, yi, xj, yj = -1, -1, -1, -1
    
    # Dipthongs are insert at row 1, columns [0.5, 2.5, 4.5]
    if(x in dipthongs):
        xi = 1
        xj = get_dipthongs_j(x)
    
    if(y in dipthongs):
        yi = 1
        yj = get_dipthongs_j(y)    
    
    for i in range(3):
        for j in range(6):
            if(x == table[i][j]):
                xi = i; xj = j;
            if(y == table[i][j]):
                yi = i; yj = j;

#     return ( (2*(xi-yi))**2 + (xj-yj)**2 )**0.5
    return  min(((2*(xi-yi))**2 + (xj-yj)**2 )**0.5, 4.0)/5.0


# FINAL ---------------------------------------------------------------------

    
def finalc_score(x, y):
    #REF table: https://en.wikipedia.org/wiki/Thai_language#Finals
    #column: Labial, Alveolar, Palatal, Velar
    #row: Nasal, Plosive, Approximant
    table = [[] for i in range(3)]
    table[0].extend([['m^'], ['n^', 'l^'], [], ['ng^']])
    table[1].extend([['p^', 'f^'], ['t^','s^','ch^'], [], ['k^']])
    table[2].extend([['w^'], [], ['j^'], []])
    
    #default as (Glottis)
    xi = 1; xj = 4; yi = 1; yj = 4
    
    for i in range(3):
        for j in range(4):
            if(x in table[i][j]):
                xi = i; xj = j
            if(y in table[i][j]):
                yi = i; yj = j
#     return ((xi-yi)**2 + (xj-yj)**2)**0.5    
    return min(((xi-yi)**2 + (xj-yj)**2)**0.5, 3.3)/4.125
   
    
# ----------------------------------------------------------------------


# def cross_type_score(x,y):
#     return 5


# def ins_del_score(x):
#     return 5
#     if(isVow(x)):
#         return 4
#     else:
#         return 4

    
# def measure_weight(x, y):        
#     if(x == y): return 0
# #     elif(y == '' or x == ''): return ins_del_score(x)
    
#     try:
#         x = dec_phone[x]
#         y = dec_phone[y]
#     except:
#         pass
        
#     if(isVow(x) and isVow(y)):
#         return vow_score(x,y)
#     elif(isFinal(x) and isFinal(y)):
#         return finalc_score(x,y)
#     elif(isInit(x) and isInit(y)):
#         return initc_score(x,y)
#     else:
#         return cross_type_score(x,y)

In [11]:
def dist_scoring(speak, target):
    init, vowel, final = speak     
    t_final = ''
    n = 3
    if(len(final) == 0): n = 2
    
    if(len(target) == 3): t_init, t_vowel, t_final = target
    else: t_init, t_vowel = target
    
    
    i_cost = initc_score(init, t_init)
    v_cost = vow_score(vowel, t_vowel)
    f_cost = finalc_score(final, t_final)
    return i_cost, v_cost, f_cost

In [13]:
pred = load_prediction('30best-disability-dLM-online.txt')
label = load_label('text-real')
# pred = load_prediction('30best-dhealthy-dLM-online.txt')
# label = load_label('text-dhealthy')

ic_all = 0
vc_all = 0
fc_all = 0

for name in sorted(pred):
    print(name, pred[name], label[name])
    i_c, v_c, f_c = dist_scoring(pred[name], label[name])
    
    print("%.2f %.2f %.2f" % (1-i_c, 1-v_c, 1-f_c))
    print("%.2f\n" % (1-(i_c+v_c+f_c)/3))

    ic_all += i_c
    vc_all += v_c
    fc_all += f_c


print("%.2f %.2f %.2f" % (1-ic_all/len(pred), 1-vc_all/len(pred), 1-fc_all/len(pred)))

D00_000 ('ph', 'a', 'n^') ['p', 'aa']
0.80 0.80 0.23
0.61

D00_001 ('t', 'ee', 'ng^') ['c', 'e', 'ng^']
0.80 0.80 1.00
0.87

D00_002 ('n', 'ii', 't^') ['c', 'ii']
0.55 1.00 0.27
0.61

D00_003 ('k', '@@', 'ng^') ['k', '@@', 'n^']
1.00 1.00 0.52
0.84

D00_004 ('f', '@@', 't^') ['k', '@@', 'j^']
0.28 1.00 0.66
0.65

D00_005 ('n', 'ii', '') ['p', 'vva', 'ng^']
0.55 0.50 0.66
0.57

D00_006 ('n', 'ii', '') ['th', 'ee']
0.40 0.60 1.00
0.67

D00_007 ('th', 'uua', 'j^') ['th', 'uua', 'j^']
1.00 1.00 1.00
1.00

D00_008 ('p', 'a', 'j^') ['p', 'a', 'j^']
1.00 1.00 1.00
1.00

D00_009 ('n', 'ii', 'p^') ['d', 'ii']
0.80 1.00 0.20
0.67

D00_010 ('ph', 'uu', 'p^') ['m', 'u', 'ng^']
0.40 0.80 0.23
0.48

D00_011 ('m', 'ii', '') ['m', 'ii']
1.00 1.00 1.00
1.00

D00_012 ('ph', 'iia', 'j^') ['ng', 'aa', 'm^']
0.20 0.36 0.31
0.29

D00_013 ('z', 'e', 'j^') ['ph', 'aa', 'j^']
0.20 0.28 1.00
0.49

D00_014 ('k', 'iia', 'k^') ['k', 'iia', 'ng^']
1.00 1.00 0.76
0.92

D00_015 ('m', 'uu', 'p^') ['z', 'uu', 'p^']
0.2

# feedback

In [14]:
measure_weight('s^', 'k^')

2.0